_（以下用缩写 sd 表示 \[Base64\](c3RhYmxlLWRpZmZ1c2lvbg==)。）_

# SD自由

本笔记本包含较多文字说明，对运行影响不大。文字说明较简略的精简版：[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1T9uU0q31wE_rWbL46_gfScFORPNjozEK)

本笔记本属于 [这个 github 项目](https://github.com/adpitacor/sd-webui-on-colab)。如果发现本笔记本的问题（如注释有误，错别字等）或运行中遇到问题，又或者想提点建议，欢迎来提 issue。

---

本笔记本按照官方指引~~并添加了_亿点_ 代码~~，从 github 上获取完全原版的 \[Base64\](c2Qtd2VidWk=)。这在带来高纯净度、高自由度的同时，也意味着需要手动配置很多东西。不过，本笔记本已经将其配置到最基础的「能用且配置一气呵成」的程度。

~~（才不会告诉你用原版是因为我找不到也不会用整合包~~

欢迎在此基础上建立使用 webui 整合包的笔记本。

---

虽说 Colab 官方已经宣布将限制 sd，但是 2023.7.7 本人亲测一个运行时（runtime）从网页端 UI 可以访问开始记时，在中等使用频率的情况下，依旧可以保持运行 3.5 小时以上而不被强制关闭。当然，这仅作参考。

关于尝试运行 sd 相关代码时会弹出警告弹窗的事，经过探索，发现弹窗仅在运行的代码块中包含特定字段时触发（只要字面上包含就会，与代码实际运行效果无关），已知的字段有 \[Base64\](c3RhYmxlLWRpZmZ1c2lvbi13ZWJ1aQ==) 和 \[Base64\](L3NkLXdlYnVpLw==)。本笔记本已尽力避开这些字段，但如果弹窗依旧出现，请点击“Cancel Execution”（取消运行）并适当修改代码后再次运行，以免（可能）被“重点关注”（但也不至于要重开）。

---

温馨提示：

- 请在 `Edit`（修改）-> `Notebook settings`（笔记本设置）中，把 `Hareware acceleterator`（硬件加速器）选项设置为 `GPU`。

- 不要用 `Run All`（全部运行），请至少粗略阅读单元格附加的文字说明再运行。

- 单元格运行中途报错都是意料之中，判断运行有问题的唯一标准是单元格结束运行而未清空输出并留下简短的运行完成提示句（对于 3. 则是单元格在未手动结束运行自动终止）。

- 单元格 3. 启动后会持续运行，若启动后希望运行其他单元格必须先手动停止该单元格的运行。

总之，希望本笔记本可以让你实现一定程度上的“SD自由”。

_（P.S. 代码中添加了注释以供学习，但有些影响代码观感。本人水平有限，对代码的解释可能不太准确或有所简化，仅供参考。）_

In [ ]:
#@title ## 0. 前置配置

#@markdown 本单元格将进行一些必要的设置，以做好安装 webui 的准备。
#@markdown
#@markdown 大致运行内容：
#@markdown
#@markdown 1. 安装「安装 webui」所需的包（主要是 venv，其他应该本来就有）；
#@markdown
#@markdown 1. 添加非 root 用户并适当放权，供运行 webui.sh 使用。

'''
此节为固定开头，后不作注释
'''
# 导入 ipython 清除输出的函数（clear 命令在此无法正常使用）
from IPython.display import clear_output as cls
# 切换目录（用 !cd 则无法生效）
%cd /content

'''
安装「安装 webui」所需的包
'''
# -y：默认同意所有操作；-qq：以静默模式运行（然而还是有输出信息）
# 默认为 root 用户，所以无需 sudo
!apt install -y -qq wget git python3 python3-venv

'''
正是因为默认 root，才有了一堆麻烦事——webui 必须以非 root 用户身份运行
所以此节添加新的非 root 用户并进行必要的放权
'''
# useradd 添加新用户（不过这个账户除了名字啥都没有）
!useradd guest0721
# 创建用户的家目录
!mkdir /home/guest0721
# 放权；-R 表示对目录及其下所有目录、文件进行设置（默认仅设置指定的目录/文件本身）
!chmod 777 /home -R
!chmod 777 /content -R
!chmod 777 /tmp -R
# 2>/dev/null 将标准错误（stderr）（2）重定向到空设备（/dev/null）（即不显示报错信息）
# /var 里有很多没法修改权限的东西，所以会报很多错（不过那些不改权也无所谓）
!chmod 777 /var -R 2>/dev/null

'''
此节为固定结尾，后不作注释
'''
# 回到默认目录，以免影响临时添加代码块进行操作时所在的目录
%cd /content
# 以 python 函数方式运行 cls 清除输出（不想清除请注释此句）
cls()
# 提示结束
!echo "Finished."

In [ ]:
#@title ## 1. 预运行 webui.sh

#@markdown 本单元格将预运行 `webui.sh` 拉取 webui 包并进行一些设置，确保接下来本地的 `webui.sh` 可以顺利运行并开启 webui 服务。
#@markdown
#@markdown 大致运行内容：
#@markdown
#@markdown 1. 获取 github 上的 webui.sh 并在本地运行（只能做到拉取 webui 包）；
#@markdown
#@markdown 1. 调整 venv 并为 launch.py 启用 --share 命令行参数。

from IPython.display import clear_output as cls
%cd /content

'''
# 需要用 su guest0721 切换到刚刚创建的非 root 用户才能运行 webui.sh
# 但是 su guest0721 并不是把当前单元格所“登录”的用户从 root 变为 guest0721，而是会以 guest0721 的身份开启一个交互式“终端”
# 这就意味着需要手动输入命令——或者，把标准输入（stdin）重定向为存有需要执行的命令的文件（即从文件读取命令并运行）
# 此节将创建出那样的文件（由于文件仅临时创建、使用，故存至 /tmp）
'''
# “右向的单箭头（>）和双箭头（>>）+文件名”都表示将命令的标准输出（stdout）重定向至文件（即让命令输出到文件中）（若文件不存在则先创建之）
# 区别在于，单箭头为（清空原内容并直接）写入，双箭头为（在文件末尾）追加
# 所以先用单箭头统一创建或清空文件，再用双箭头追加写入
# 只有“右箭头+文件名”的语句也是成立的（如下）
# “箭头+文件名”语句可以放在命令的开头前或结尾后，即 > a echo "a" 和 echo "a" > a 一样
# 批量操作时采取前置更美观
!>  /tmp/cmd.txt
# bash -c "..." 的省略号部分为官方提供的预运行 webui.sh 的指令
# wget中，-q 表示静默模式，-O FILE 表示将下载的内容输出到指定的文件 FILE 中，而 - 作 FILE 表示把标准输出当作指定的文件（即把下载内容输出到标准输出）
# -qO- 相当与 -q -O -
# 而 bash <(wget ...)（这个语法中 <( 中间不能加空格）则表示将 wget 的标准输出作为 bash 的标准输入，有点像管道（wget ... | bash）但有一点差别
# 概括起来就是：从 github 上获取 webui.sh 并在本地用 bash 运行
# 但由于 <(...) 语法本身不能被 su 打开的“终端”识别，所以只好套个 bash 再跑
# bash -c "CMD" 表示启动 bash 并在运行完 CMD 后退出 bash
# 可以看到 URL 经过了处理，把字段割开了，而根据规则，'a''b' 会被合并为 'ab'，所以没影响
!>> /tmp/cmd.txt echo 'bash -c "bash <(wget -qO- https://raw.githubusercontent.com/AUTOMATIC1111/sta''ble-diff''usion-web''ui/master/webui.sh)"'
# 预运行 webui.sh 完成后，仅仅是拉取了 webui 包，还有一些东西要先搞好
# 改个名先
!>> /tmp/cmd.txt echo "mv sta""ble-dif""fusion-web""ui webui"
!>> /tmp/cmd.txt echo "cd webui"
# 由于 venv 里没有 pip，也没有 ensurepip，所以需要用 get-pip.py 安装 pip
!>> /tmp/cmd.txt echo "wget -qO /tmp/get-pip.py https://bootstrap.pypa.io/get-pip.py"
# 要用 venv 里的 python3 跑 get-pip.py，才能把 pip 装到 venv 里
!>> /tmp/cmd.txt echo "venv/bin/python3 /tmp/get-pip.py"
# venv 需要通过运行 venv 模块来 activate
# 同样由于 venv 里没有 ensurepip，所以要加 --without-pip 跳过 ensurepip 的运行
!>> /tmp/cmd.txt echo "python3 -m venv venv/ --without-pip"
# 设置 launch.py 命令行参数
# --share 表示创建公共链接
!>> /tmp/cmd.txt echo "sed -i 's/#export COMMANDLINE_ARGS=\"\"/export COMMANDLINE_ARGS=\"--share\"/g' webui-user.sh"
!>> /tmp/cmd.txt echo "exit"

'''
运行！
'''
# “左键头（<）+文件名”表示将命令的标准输入重定向为文件（即让命令从文件读入）
!<  /tmp/cmd.txt su guest0721

%cd /content
cls()
!echo "Terminated."

In [ ]:
#@title ## 2.（可选，但不完全可选）下载大模型

#@markdown 本单元块旨在预先下载大模型。
#@markdown
#@markdown 虽说就算不额外下载，程序也会帮你下载默认的 sd 模型，但你应该不会想用默认模型。
#@markdown
#@markdown 当然，要安装 LoRA、插件之类的也请在这里先安装。
#@markdown
#@markdown 大模型上哪里找？你可以逛逛 [Civitai](https://civitai.com)，找到心仪的模型后也可以到 [huggingface](https://huggingface.co)（国内可访问）上下载（若能查到）。
#@markdown
#@markdown 请尝试右键点击下载按钮并在右键菜单中点“复制链接”以获取下载直链（直链访问后会直接返回文件作为结果，请确认这一点）。
#@markdown
#@markdown （顺带一提，本人对着网图抄作业（指用与别人的图相同的参数自己跑图，可以用 webui 中的 PNG Info 做到）从来没有成功过，如果有成功的欢迎交流）
#@markdown
#@markdown 嫌麻烦？那就上B站看看 [秋叶大佬](https://space.bilibili.com/12566101) 或者其他 up 的推荐吧。
#@markdown
#@markdown 实在不想挑就无脑选 anything 系列（可在 Civitai/huggingface 上找到）。~~六边形战士~~

from IPython.display import clear_output as cls
# 切换到模型存放目录（大模型，LoRA 等都放这，请自己翻翻目录看看什么放哪里）
%cd /content/webui/models

# 使用 aria2c 下载模型文件（注意 apt install 的是 aria2 而不是 aria2c）
!apt install -y -qq aria2

'''
在 huggingface 上下载时，链接会被重定向，导致下载的文件的文件名变成十六进制字符串
所以要手动设置下载的文件名
由于一次要下载多个文件，都得设置，所以把链接和设置都写到文件里
对于每个链接，格式大概是：
[链接]
  out=[文件名]
这里随便选了几个模型装
'''
!>  /tmp/uri.txt
!>> /tmp/uri.txt echo "https://civitai.com/api/download/models/30163"
!>> /tmp/uri.txt echo "  out=AnythingV5Ink_v5PrtRE.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/ckpt/anything-v4.5-vae-swapped/resolve/main/anything-v4.5-vae-swapped.safetensors"
!>> /tmp/uri.txt echo "  out=anything-v4.5-vae-swapped.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/PWB/starlake/resolve/main/starlake-3.5.safetensors"
!>> /tmp/uri.txt echo "  out=starlake-3.5.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors"
!>> /tmp/uri.txt echo "  out=Counterfeit-V3.0_fp16.safetensors"
'''
下载模型文件
'''
# -i FILE 表示从 FILE 中读取需要下载的文件的 URL 及相关设置，--summary-interval=5 表示每过 5 秒显示当前下载进度
# --allow-overwrite=true 表示允许覆写，-d PATH 表示把文件下载到 PATH 中
!aria2c -i /tmp/uri.txt --summary-interval=5 -x 3 --allow-overwrite=true -d Stable-diffusion

'''
下载通用 VAE
'''
!>  /tmp/uri.txt
!>> /tmp/uri.txt echo "https://huggingface.co/a1079602570/animefull-final-pruned/resolve/main/animevae.pt"
!>> /tmp/uri.txt echo "  out=animevae.pt"
!aria2c -i /tmp/uri.txt --summary-interval=5 -x 3 --allow-overwrite=true -d VAE

'''
要安装其他诸如 LoRA、插件之类的请在此手动添加代码安装
'''

%cd /content
cls()
!echo "Finished."

In [ ]:
#@title ## 3. 启动 webui

#@markdown 本单元格将运行本地的 webui.sh，正式启动 webui。
#@markdown
#@markdown 请耐心等待，等到 `Running on public URL: https://*.gradio.live` 一行输出后，复制链接（此即公链）并访问，即可打开 webui。国内也可访问此公链。
#@markdown
#@markdown 初次运行需要安装所需的模块，用时相对较长，属正常现象。
#@markdown
#@markdown 注意，请在运行前确认所需的模型、插件等都已安装。

from IPython.display import clear_output as cls
%cd /content/webui

!>  /tmp/cmd.txt
!>> /tmp/cmd.txt echo "./webui.sh"
!>> /tmp/cmd.txt echo "exit"

!<  /tmp/cmd.txt su guest0721

%cd /content
cls()
!echo "Terminated."

In [ ]:
#@title ## 4.（可选）把所有生成出来的图片打包带走

#@markdown 本单元格提供了一种下载已生成的图片的方式。
#@markdown
#@markdown 单元格运行完成后会输出一个链接，访问该链接即可在第三方文件托管平台（[gofile.io](https://gofile.io/)，国内可访问）下载到图片压缩包。
#@markdown
#@markdown 当然，你也可以在 webui 中保存图片，而且那样做风险或许小些——要是笔记本连接真的被强制掐断，文件就都没了。

from IPython.display import clear_output as cls
%cd /content

'''
把图片打成压缩包
'''
# -f 强制删除，不管文件是否存在都不提示
!rm -f images.zip
# -r 将指定要压缩的目录及其下的所有目录/文件都添加到压缩包
!zip -r images.zip webui/outputs

'''
上传压缩包到第三方文件托管平台
'''
# subprocess.check_output 取一个列表为参数，可以大致理解成将列表各项以空格相隔进行合并得到命令行命令，运行，并返回其标准输出的结果
from subprocess import check_output
# json.loads 将 json 字符串转为 Python 字典对象
from json import loads
try:
  # 获取最佳服务器
  res = loads(check_output(["curl", "https://api.gofile.io/getServer"]))
  assert res["status"] == "ok"
  srv = res["data"]["server"]
  # 上传文件并得到其下载链接（非直链）
  res = loads(check_output(["curl", "-F", "file=@images.zip", f"https://{srv}.gofile.io/uploadFile"]))
  assert res["status"] == "ok"
  url = res["data"]["downloadPage"]
except:
  ok = False
else:
  # 这里的 else 块会在 try 块正常完成运行后被执行
  ok = True

%cd /content
cls()
if ok: print("link:", url)
else:  print("Failed to upload images.zip")